In [1]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import pandas as pd

from sklearn.datasets import load_iris
import numpy as np

In [2]:
torch.manual_seed(2)

# Load Iris Dataset

In [12]:
iris = load_iris()
X = iris.data
Y = iris.target

Use train_test_split to split data into train and validation set

In [13]:
from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [14]:
x.shape, y.shape, x_val.shape, y_val.shape

((100, 4), (100,), (50, 4), (50,))

In [15]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [16]:
x_train.shape

(100, 4)

Define validation data as a Pytorch Tensor

In [17]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

Put Data through DataLoader, so we can use batches

In [18]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train)
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [19]:
data_set=Data()

In [20]:
trainloader=DataLoader(dataset=data_set,batch_size=64)

In [21]:
data_set.x[1:10]

tensor([[7.6000, 3.0000, 6.6000, 2.1000],
        [5.6000, 3.0000, 4.5000, 1.5000],
        [5.1000, 3.5000, 1.4000, 0.2000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [5.0000, 3.5000, 1.3000, 0.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.0000, 2.0000, 3.5000, 1.0000]])

In [22]:
data_set.y[1:10]

tensor([2, 1, 0, 2, 1, 0, 0, 0, 1])

In [23]:
data_set.x.shape, data_set.y.shape

(torch.Size([100, 4]), torch.Size([100]))

# Build Model and train it

Build softmax Classifier

In [24]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [25]:
input_dim=4     # how many Variables are in the dataset
hidden_dim = 25 # hidden layers
output_dim=3    # number of classes
input_dim

4

In [27]:
# Instantiate model
model=Net(input_dim,hidden_dim,output_dim)

View the size of the model parameters:

In [28]:
print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

W: torch.Size([25, 4])
b torch.Size([25])


Loss function

In [29]:
criterion=nn.CrossEntropyLoss()

Optimizer

In [30]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [31]:
n_epochs=1000
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    for x, y in trainloader:
      

        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss, da Cross Entropy benutzt wird muss ich in den loss Klassen vorhersagen, 
        # also Wahrscheinlichkeit pro Klasse. Das mach torch.max(y,1)[1])
        loss=criterion(z,y)
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        
        loss_list.append(loss.data)
        
        
        #print('epoch {}, loss {}'.format(epoch, loss.item()))

In [33]:
z[0:10]

tensor([[  9.3748,   4.2373, -13.0650],
        [ -5.3994,   1.9242,   3.5881],
        [ -1.6497,   3.4716,  -1.5646],
        [ -5.3590,   1.9075,   3.5724],
        [  1.0837,   4.4060,  -5.1889],
        [ -2.1853,   3.1301,  -0.6984],
        [ -2.4463,   3.2549,  -0.5641],
        [  7.8511,   4.3348, -11.7414],
        [ -1.5189,   3.7685,  -2.0043],
        [ -3.1103,   2.9239,   0.4084]], grad_fn=<SliceBackward>)

Each row represents a row from the original dataset. Each column represents a class. The first column represents the class 0, the second column class 1 and the third column class 2. The highest value for each row represents which class the model would put each row. For instance, the highest value in the first row is 9.3748, hence the predicted class is 0.  

Check model on Validation Set

In [34]:
z=model(x_val)

In [35]:
yhat=torch.max(z.data,1)
yhat

(tensor([3.0738, 9.2892, 7.5300, 2.9072, 3.2767, 9.0327, 3.9651, 3.4732, 2.4689,
         3.8023, 3.0332, 8.8700, 9.4850, 8.8601, 9.3904, 3.0966, 6.2266, 3.6219,
         2.8405, 6.1747, 8.6625, 2.8811, 8.7867, 5.9667, 3.6716, 4.4869, 5.8407,
         6.0122, 8.6818, 8.5801, 9.6922, 9.8235, 3.9622, 8.9117, 9.0107, 4.3871,
         3.5250, 9.2316, 9.3840, 9.7249, 5.1719, 3.1824, 3.4399, 9.6400, 9.4289,
         3.8427, 2.6601, 4.3333, 3.8105, 5.8017]),
 tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
         2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
         1, 2]))

In [36]:
y_val

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2])